# Map residential building vulnerability curves to individual buildings

This notebook maps the modified building vulnerability relations to the augmented NEXIS data for SWHA. 

The building vulnerability curves were provided by the Cyclone Testing Station (James Cook University), while the exposure data has been augmented to include U.S. Navy block houses in Exmouth, which were previously attributed with tile roofs. 

In [1]:
%matplotlib inline

from os.path import join as pjoin
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
project_path = "X:/georisk/HaRIA_B_Wind/projects/dfes_swha/data"
exp_file = "processed/exposure/WA_Residential_Exposure_2018.csv"
vul_file = "processed/vulnerability/swha_domestic_wind_vul_curves.csv"

exp_df = pd.read_csv(pjoin(project_path, exp_file))
vul_df = pd.read_csv(pjoin(project_path, vul_file))

In [3]:
exp_df['YEAR_BUILT'].unique()

In [4]:
vul_df.head()

In [5]:
wallType = vul_df['Roof material'].unique()
roofType = vul_df['Wall material'].unique()
ageClass = vul_df['Age'].unique()
regions = vul_df['Region'].unique()
print(wallType)
print(roofType)
print(ageClass)
print(regions)

In [6]:
walldict = {'Double Brick': ['Double Brick'],
            'Fibro / asbestos cement sheeting':['Fibro / asbestos cement sheeting'],
            'Brick Veneer':['Brick Veneer'],
            'Timber': ['Timber']}
roofdict = {'Metal Sheeting':['Metal Sheeting'],
            'Fibro / asbestos cement sheeting':['Fibro / asbestos cement sheeting'],
            'Tiles': ['Tiles'],
            'Concrete': ['Concrete']}
agedict = {'1960 to 1979':['1891 - 1913', '1914 - 1946', '1947 - 1961', '1962 - 1981'],
           '1980 to present':['1982 - 1996', '1997 - present']}

# Regions are mapped based on LGA_NAME
regiondict = {'Perth': [],
              'Geraldton': [],
              'Carnarvon': ['Carnarvon (S)', 'Upper Gascoyne (S)', 'Shark Bay (S)'],
              'Exmouth': ['Exmouth (S)'],
              'Karratha': ['Roebourne', 'Karratha (C)', 'Ashburton (S)'],
              'Port Hedland': ['East Pilbara (S)', 'Port Hedland (T)', 'South Hedland'],
              'Broome': ['Broome (S)', 'Wyndham-East Kimberley (S)', 'Halls Creek (S)', 'Derby-West Kimberley (S)']}

newexpdf = exp_df.copy()

In [7]:
print(newexpdf['YEAR_BUILT'].isin(agedict['1960 to 1979']).sum())
print(newexpdf['ROOF_TYPE'].isin(roofdict['Concrete']).sum())

In [8]:
nbld = 0
for idx, row in vul_df.iterrows():
    vfId = row['vulnerabilityFunctionID']
    age = row['Age']
    roof = row['Roof material']
    wall = row['Wall material']
    region = row['Region']
    print("{0}, {1}, {2}, {3}, {4}".format(vfId, age, roof, wall, region))
    agefilter = newexpdf['YEAR_BUILT'].isin(agedict[age])
    rooffilter = newexpdf['ROOF_TYPE'].isin(roofdict[roof])
    wallfilter = newexpdf['WALL_TYPE'].isin(walldict[wall])
    regionfilter = newexpdf['LGA_NAME'].isin(regiondict[region])
    print("Ages: {0}".format(agefilter.sum()))
    print("Roofs: {0}".format(rooffilter.sum()))
    print("Walls: {0}".format(wallfilter.sum()))
    print("Regions: {0}".format(regionfilter.sum()))

    print((agefilter & rooffilter & wallfilter & regionfilter).sum())
    nbld += (agefilter & rooffilter & wallfilter & regionfilter).sum()

    newexpdf.at[(agefilter & rooffilter & wallfilter & regionfilter), 'WIND_VULNERABILITY_FUNCTION_ID'] = vfId
    newexpdf.at[(regionfilter), 'WIND_REGION_CLASSIFCATION'] = region
    newexpdf.at[(newexpdf['YEAR_BUILT'].isin(agedict[age])),'AGE'] = age

    
print(nbld)

In [9]:
newexpdf[['WIND_VULNERABILITY_FUNCTION_ID','LGA_NAME', 'ROOF_TYPE', 'WALL_TYPE', 'AGE']]

In [10]:
newexpdf.to_csv(pjoin(project_path, "processed/exposure/WA_TILES_Residential_Wind_Exposure_notiles.csv"))

In [11]:
oldvfdf = newexpdf[newexpdf['WIND_VULNERABILITY_FUNCTION_ID'].str.startswith('dw')]
oldvfdf

In [12]:
oldvfdf.groupby([ 'WIND_REGION_CLASSIFCATION','ROOF_TYPE', 'WALL_TYPE','AGE'])['LID'].count()

In [13]:
outputdf = oldvfdf.groupby([ 'WIND_REGION_CLASSIFCATION','ROOF_TYPE', 'WALL_TYPE','AGE'])['LID'].count()

In [15]:
outputdf.to_excel(pjoin(project_path, "processed/exposure/WA_building_combinations.xlsx"))